11314

In [38]:
import pandas as pd
from sklearn.datasets import fetch_20newsgroups
import os,io

files = os.listdir(os.getcwd()+'/data/spao_reviews/')
result_df =pd.DataFrame()
for file in files : 
    df=pd.read_parquet(os.getcwd()+'/data/spao_reviews/' + file)
    
    result_df = pd.concat([result_df,df])
print(len(result_df))
result_df.reset_index(drop=True,inplace=True)

42193


In [39]:
result_df['rev_cont'][0]
result_df['clean_Doc'] = result_df['rev_cont'].str.replace("[^a-zA-Z가-힣]", " ") #특수문자제거
result_df['clean_Doc'] = result_df['clean_Doc'].apply(lambda x: ' '.join([w for w in x.split() if len(w)>2])) 
#영어는 3글자 한글은 2글자 이상 제거

result_df['clean_Doc'][0]

c:\users\jeong_jaekeun\appdata\local\programs\python\python37\lib\site-packages\ipykernel_launcher.py:2: FutureWarning: The default value of regex will change from True to False in a future version.
  


'testest'

In [52]:
from nltk.corpus import stopwords
stop_words = stopwords.words('english') # NLTK로부터 불용어를 받아옵니다.
tokenized_doc = result_df['clean_Doc'].apply(lambda x: x.split()) # 토큰화
tokenized_doc = tokenized_doc.apply(lambda x: [item for item in x if item not in stop_words])

In [53]:
tokenized_doc[4]

['소재도', '원하던', '디자인대로', '상품도', '나왔습니다']

In [54]:
detokenized_doc = []
for i in range(len(result_df)):
    t = ' '.join(tokenized_doc[i])
    detokenized_doc.append(t)

result_df['clean_doc'] = detokenized_doc

In [55]:
result_df['clean_doc'][4]

'소재도 원하던 디자인대로 상품도 나왔습니다'

In [56]:
from sklearn.feature_extraction.text import TfidfVectorizer

vectorizer = TfidfVectorizer(stop_words='english', 
max_features= 1000, # 상위 1,000개의 단어를 보존 
max_df = 0.5, 
smooth_idf=True)

X = vectorizer.fit_transform(result_df['clean_doc'])
X.shape # TF-IDF 행렬의 크기 확인

(42193, 1000)

In [61]:
from sklearn.decomposition import TruncatedSVD
svd_model = TruncatedSVD(n_components=10, algorithm='randomized', n_iter=100, random_state=122)
svd_model.fit(X)
len(svd_model.components_)

10

In [62]:
terms = vectorizer.get_feature_names() # 단어 집합. 1,000개의 단어가 저장됨.

def get_topics(components, feature_names, n=5):
    for idx, topic in enumerate(components):
        print("Topic %d:" % (idx+1), [(feature_names[i], topic[i].round(5)) for i in topic.argsort()[:-n - 1:-1]])
get_topics(svd_model.components_,terms)

Topic 1: [('좋아요', 0.9846), ('따뜻하고', 0.10937), ('편하고', 0.05805), ('사이즈', 0.03849), ('사이즈도', 0.03727)]
Topic 2: [('예뻐요', 0.84805), ('이뻐요', 0.2357), ('따뜻하고', 0.20488), ('같아요', 0.17084), ('사이즈', 0.14213)]
Topic 3: [('이뻐요', 0.87734), ('따뜻하고', 0.12851), ('좋습니다', 0.10754), ('사이즈', 0.08818), ('같아요', 0.07163)]
Topic 4: [('따뜻하고', 0.57397), ('좋습니다', 0.41731), ('귀여워요', 0.25568), ('같아요', 0.18125), ('사이즈', 0.13841)]
Topic 5: [('따뜻하고', 0.69846), ('이뻐요', 0.08438), ('예뻐요', 0.08151), ('편해요', 0.01654), ('부드러워요', 0.0119)]
Topic 6: [('귀여워요', 0.75488), ('따뜻하고', 0.26313), ('이뻐요', 0.04969), ('예뻐요', 0.03918), ('너무너무', 0.0177)]
Topic 7: [('좋습니다', 0.69049), ('귀여워요', 0.32532), ('예뻐요', 0.11362), ('이뻐요', 0.08629), ('편하고', 0.07069)]
Topic 8: [('사이즈', 0.80322), ('맞아요', 0.152), ('귀여워요', 0.10015), ('따뜻하고', 0.06606), ('적당하고', 0.02798)]
Topic 9: [('따뜻해요', 0.4204), ('마음에', 0.35539), ('편하고', 0.35013), ('들어요', 0.34313), ('사이즈도', 0.18859)]
Topic 10: [('따뜻해요', 0.58789), ('편하고', 0.39279), ('생각보다', 0.14202), ('부드럽고', 0.1124), (